In [1]:
import sys
sys.path.append('/Users/darapoludova/Documents/courses/LemonsMarket')
import matplotlib. pyplot as plt

import numpy as np
import pandas as pd
import torch
import torch.nn as nn

from config import *
from game import StaticGame
from agents import EpsGreedyAgent, BolzmanAgent, DQNAgent

In [2]:
device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda')
    
print(device)

cpu


In [3]:
device

device(type='cpu')

In [4]:
class SimpleModel(nn.Module):
    def __init__(self, n_features, hidden_size=16):
        super().__init__()
        self.linear_1 = nn.Linear(n_features, hidden_size)
        self.linear_2 = nn.Linear(hidden_size, hidden_size)
        self.linear_3 = nn.Linear(hidden_size, 1)
        
    def forward(self, features):
        output = nn.functional.relu(self.linear_1(features))
        output = nn.functional.relu(self.linear_2(output))
        output = self.linear_3(output)
        return output

In [5]:
# define game and agents

seller_model = SimpleModel(10)
opt_seller = torch.optim.Adam(seller_model.parameters(), lr=1e-3)

buyer_model = SimpleModel(10)
opt_buyer = torch.optim.Adam(buyer_model.parameters(), lr=1e-3)

game = StaticGame()
agent_seller = DQNAgent('seller', seller_model, 1.0)
agent_buyer = DQNAgent('buyer', buyer_model, 1.0)

In [6]:
# play one game to collect data
def play_one_game(game, agent_seller, agent_buyer, n_iters=100):
    
    for iter_num in range(n_iters):
        game.reset()

        seller_action = agent_seller.choose_action(game.car_type)
        game.step('seller', seller_action)
        buyer_action = agent_buyer.choose_action(seller_action)
        game.step('buyer', buyer_action)
            
        agent_seller.observe_reward(game.reward('seller'))
        agent_buyer.observe_reward(game.reward('buyer'))
        
        # remember history for models fitting
        for agent in (agent_seller, agent_buyer):
            agent.observe_game_result((game.car_type, seller_action, buyer_action))

In [7]:
play_one_game(game, agent_seller, agent_buyer)

In [ ]:
# train & evaluate model with callback
# save model ?

In [58]:
# get train / val datasets

X, y = agent_seller.get_train_data()
n_examples = len(y)
test_idx = np.random.choice(n_examples, size=int(n_examples * 0.1), replace=False)
train_idx = list(set(range(n_examples)) - set(test_idx))

y = np.array(y)
X_train, X_test = X[train_idx], X[test_idx]
y_train, y_test = y[train_idx], y[test_idx]

AttributeError: 'list' object has no attribute 'shape'

In [54]:
X[list(set(range(n_examples)) - set(test_idx))]

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  1,  0,  0,  0,  0,  1,  0],
       [ 0,  0,  0,  1,  0,  1,  0,  0,  1,  0],
       [ 0,  0,  0,  1,  1,  1,  0,  0,  0,  1],
       [ 0,  0,  0,  2,  1,  1,  0,  0,  1,  1],
       [ 0,  0,  0,  2,  1,  1,  1,  0,  1,  0],
       [ 0,  0,  0,  2,  2,  1,  1,  0,  1,  1],
       [ 0,  0,  0,  2,  2,  1,  2,  0,  1,  0],
       [ 0,  0,  0,  2,  2,  2,  2,  0,  0,  0],
       [ 0,  1,  0,  2,  2,  2,  2,  0,  1,  1],
       [ 0,  1,  0,  2,  2,  2,  2,  1,  1,  1],
       [ 0,  1,  0,  2,  2,  2,  3,  1,  1,  0],
       [ 0,  1,  0,  2,  2,  3,  3,  1,  1,  0],
       [ 0,  1,  0,  2,  3,  3,  3,  1,  1,  1],
       [ 0,  1,  0,  2,  3,  3,  4,  1,  1,  0],
       [ 0,  1,  0,  2,  3,  4,  4,  1,  1,  1],
       [ 0,  1,  0,  2,  3,  4,  4,  2,  1,  0],
       [ 0,  1,  0,  2,  3,  6,  4,  2,  1,  1],
       [ 0,  1,  0,  2,  3,  6,  5,  2,  1,  0],
       [ 0,  1,  0,  2,  4,  6,  5,  2,  1,  0],
       [ 0,  1,  0, 

In [34]:
test_mask

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.])

In [9]:
test = torch.tensor(np.array([[0] * 10, [1] * 10]), dtype=torch.float32)

In [10]:
seller_model(test).data.numpy().ravel().shape

(2,)

In [11]:
def compute_loss(reference, prediction):
    return torch.mean((prediction - reference) ** 2)